# Pose Estimation & Squat Counter

Pose estimation refers to a general problem in computer vision techniques that detect human figures in images and videos, so that one could determine, for example, where someone’s elbow shows up in an image. It is important to be aware of the fact that pose estimation merely estimates where key body joints are and does not recognize who is in an image or video. The pose estimation models takes a processed camera image as the input and outputs information about keypoints. The keypoints detected are indexed by a part ID, with a confidence score between 0.0 and 1.0. The confidence score indicates the probability that a keypoint exists in that position. 

### 0. Install and Import Dependencies


Install the dependices used for developing this project

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib


Import the libraries that will be used in the project 

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2


### 1. Load the model 

Load the MoveNet model from tensorflow hub 

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

### 2. Draw Keypoints

In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    """ Detect and draw the keypoints
    
    Parameters
    -------------
    frame : Array 
          The frame from the web cam  of shape  480*640*3
    keypoints: Array
            The detected keypints with the scores     
    confidence_threshold: Float
            The threshold used to compare the keypoints score to it.
    
    Outputs 
    ------------
    None 
    
    """
    # extract the shape from the frame 
    y, x, _ = frame.shape
    
    # Scale the detected keypoints 
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
     
    # looping over all the keypoints and draw circle on it, if it passes the threshold
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)
    

### 3. Draw Edges 

In [5]:
# The differnet connections between the coordinates
EDGES = {
    (0, 1): (255,0,255),
    (0, 2): (0,255,255),
    (1, 3): (255,0,255),
    (2, 4): (255,0,255),
    (0, 5): (255,0,255),
    (0, 6): (0,255,255),
    (5, 7): (255,0,255),
    (7, 9): (255,0,255),
    (6, 8): (0,255,255),
    (8, 10): (0,255,255),
    (5, 6): (255,255,0),
    (5, 11): (255,0,255),
    (6, 12): (0,255,255),
    (11, 12): (255,255,0),
    (11, 13): (255,0,255),
    (13, 15): (255,0,255),
    (12, 14): (0,255,255),
    (14, 16): (0,255,255)
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    """ Drawing the connection lines between the keypoints.
    
    Parameters
    -------------
    frame : Array 
          The frame from the web cam  of shape  480*640*3
    keypoints: Array
            The detected keypints with the scores     
    edges: dict
            The different connections between the keypoints
    confidence_threshold: Float
            The threshold used to compare the keypoints score to it.
            
    Outputs 
    ------------
    None
    
    """
    
    # extract the shape from the frame 
    y, x, c = frame.shape
    
    # Scale the detected keypoints 
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    # Looping over the edges and draw them if the scores of the two keypoints passing the threshold
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

### 4. Detect Squats 

In [7]:
def count_squats(frame, keypoints, confidence_threshold, squat_counter, count_bool ):
    """ Counting the number of squats
    
    Parameters
    -------------
    frame : Array 
          The frame from the web cam  of shape  480*640*3
    keypoints: Array
            The detected keypints with the scores     
    confidence_threshold: Float
            The threshold used to compare the keypoints score to it.
    squat_counter: Int 
            This variable is used to save the number of squats    
    count_bool: bool
            It is used to detect new squats so the counter would incrment the squat counter

    Outputs 
    ------------
    squat_counter: Int
            This variable is used to save the number of squats
    count_bool: bool
            It is used to detect new squats so the counter would incrment the squat counter
    """
    
    
    # extract the shape of the frame 
    y, x, c = frame.shape
    # scale the keypoints 
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    # extract the y,x,c for the left and right hip and left and right knee 
    y_left_hip,x_left_hip,c_left_hip = shaped[11]
    y_right_hip,x_right_hip,c_righ_hip = shaped[12]
    
    y_left_knee,x_left_knee,c_left_knee = shaped[13]
    y_right_knee,x_right_knee,c_righ_knee = shaped[14]

    # check that if it is a new squat 
    if count_bool:
        # check that the key points passes the threshold 
        if (c_left_hip > confidence_threshold) & (c_righ_hip > confidence_threshold) & (c_left_knee > confidence_threshold) & (c_righ_knee > confidence_threshold):  
            # check if you are in the squat position 
            if (y_left_hip >= y_left_knee -20 )  & (y_right_hip >= y_right_knee-20):
                squat_counter = squat_counter + 1
                count_bool = False
    else:
        # check if you are out of the squat position
        if (y_left_hip + 50 <= y_left_knee )  & (y_right_hip + 50 <= y_right_knee ):
            count_bool = True

    
    # print the number of squats on the current frame 
    bottomLeftCornerOfText = (10,400)
    fontScale = 2
    fontColor = (225,0,0)
    lineType = 2
    
    cv2.putText(frame,
                'Number of Squats'+' '+str(squat_counter), 
                bottomLeftCornerOfText, 
                1, 
                fontScale,
                fontColor,
                lineType)    
    
    bottomLeftCornerOfText = (10,440)
    cv2.putText(frame,
                'Calories burnt '+' '+str(squat_counter*0.32)+ 'Kcal', 
                bottomLeftCornerOfText, 
                1, 
                fontScale,
                fontColor,
                lineType)    


    return squat_counter, count_bool           

### 5. Make Detection 

In [8]:
# acessing web cam with openCV
cap = cv2.VideoCapture(0)
squat_counter = 0
count_bool = True 
frame_counter = 0
while cap.isOpened():
    
    # counting the frames to be used in the squat detector 
    frame_counter = frame_counter + 1   
    
    # read the frames from the web cam
    ret, frame = cap.read()
    
    # reshape image to be in proper format of the model
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)

    # setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # make predicitions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    
    # get the results
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    # rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.35)
    draw_keypoints(frame, keypoints_with_scores, 0.35)
    
    # squat counting 
    squat_counter, count_bool = count_squats(frame, keypoints_with_scores, 0.35, squat_counter, count_bool)        
    
    # Show the rendered images 
    cv2.imshow('MoveNet Lighting', frame)
    
    # Stop reading if q is pressed
    if cv2.waitKey(10) & 0xff == ord('q'):
        break 

        
cap.release()
cv2.destroyAllWindows()
        

[[[[0.6840428  0.5527301  0.5836677 ]
   [0.6241096  0.62245923 0.3835004 ]
   [0.63577664 0.48867226 0.46483675]
   [0.6807646  0.70086277 0.516172  ]
   [0.6872697  0.42421937 0.48242253]
   [0.87069005 0.7904166  0.10309696]
   [0.8798249  0.32679248 0.07911959]
   [0.8564374  0.8603663  0.00974765]
   [0.6207938  0.42968518 0.04147208]
   [0.7220762  0.7269329  0.05055588]
   [0.714226   0.43366235 0.01586255]
   [0.7146558  0.68590254 0.01219729]
   [0.6831602  0.4282207  0.01627809]
   [0.8661159  0.8608576  0.00691494]
   [0.61375767 0.41140723 0.01260599]
   [0.77901596 0.7380668  0.03804553]
   [0.6882274  0.3262884  0.00381345]]]]
[[[[0.6820399  0.6192878  0.55325663]
   [0.62217534 0.6952981  0.5404918 ]
   [0.61695635 0.54832226 0.63956136]
   [0.6829556  0.7679507  0.6079084 ]
   [0.6640356  0.4764024  0.64150167]
   [0.84298193 0.89501804 0.1695598 ]
   [0.8783197  0.3193054  0.31949326]
   [0.9672824  0.9783071  0.01221436]
   [0.95559955 0.2566758  0.01282397]
   [0.839

Other fitness movments as push up, lunges and squat jumping by adapting the consscuitve change in the keypoints and if they met a certain conditions it would be counted 